# US Vehicles Pricing Regression 

Using vehicle features such as engine, fuel, mileage to predict the selling price, using the target variable MSRP.

Dataset Source: https://www.kaggle.com/datasets/CooperUnion/cardataset

### Load Data

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import math

from scipy.stats import boxcox
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures 
from sklearn.preprocessing import OneHotEncoder

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# import the csv file 
inpath = r'/Users/akhilmathur/Desktop/python_files/datasets/car_data.csv'
df = pd.read_csv(inpath)
df.head(10)

In [ ]:
# Set the target and feature columns 
trgt_col = 'MSRP'
ftr_cols = df.columns.to_list()[:-1]

### Exploratory Data Analysis

#### Descriptive Statistics 

In [ ]:
df.info()
df.describe().round(2)

#### Data Cleaning - Missing and Duplicate Values 

In [ ]:
# remove duplicate rows in data
n_dup = df.duplicated().sum()
print("{} duplicate rows to be dropped.".format(n_dup))
df.drop_duplicates(keep = 'first', inplace = True) 

In [ ]:
# count missing values in each column
print(df.isna().sum())

# use ffill to impute missing values - as similar car types present together 
df.fillna(method = 'ffill', inplace = True)
print("\nMissing values post-imputation : {}".format(df.isna().sum().sum()))

#### Correlation Analysis

In [ ]:
# check for correlations 
df_num = df.select_dtypes(exclude = 'object')
num_cols = df_num.columns.to_list()
corr_ = abs(df_num.corr())

# generate heatmap
sns.heatmap(corr_)

In [ ]:
# correlation of numeric features with target variable (MSRP) 
print(corr_.iloc[:,-1].sort_values(ascending = False))

# we remove the redundant features like popularity that have minimal impact on the MSRP target variable 
df_num.drop('Popularity', axis = 'columns', inplace = True)

#### Numerical Features Transformation 

In [ ]:
# we check the skew - based on this and the above pairplots its clear that the features are not normally distributed 
print(df_num.skew().sort_values(ascending = False))

In [ ]:
df_num_trsfd = df_num.copy()
exc_frm_trns = [trgt_col,'Year','Number of Doors']

# apply log transformation to features to reduce skew in distribution 
for col in df_num.columns:
    if col not in exc_frm_trns:
        df_num_trsfd[col] = np.log1p(df_num_trsfd[col])

# we notice the features becoming slightly less skewed - target variable was left untouched 
print(df_num_trsfd.skew().sort_values(ascending = False))
df_num_trsfd.reset_index(drop = True, inplace = True)

In [ ]:
# generate pairplot to observe features relationship with target variable (MSRP) 
sns.pairplot(df_num_trsfd)

In [ ]:
# Non-linear relationship of features visible with MSRP in above pairplot - add polynomial features 
columns_to_poly = ['Engine HP','Engine Cylinders','highway MPG', 'city mpg']
deg = 3
poly = PolynomialFeatures(deg, include_bias=False)
X_poly = poly.fit_transform(df_num_trsfd[columns_to_poly])

# Combine polynomial features with original DataFrame
df_poly = pd.concat([df_num_trsfd, pd.DataFrame(X_poly, columns=poly.get_feature_names_out(columns_to_poly))], axis=1)

# save the clean numerical df in df_num
df_num_fin = df_poly.copy()

#### Categorical Features Cleanup and Encoding

In [ ]:
# get count of unique values for each categorical value 
df_cat = df.select_dtypes(include = 'object')
for col_ in df_cat.columns:
    print("{}:{}".format(col_,df_cat[col_].nunique()))

In [ ]:
for col_ in df_cat.columns:
    print("{}:".format(col_))
    print(list(df_cat[col_].unique()))

In [ ]:
# Consolidate the engine fuel feature categories
fuel_map = {"petrol" : ['premium unleaded (required)', 'regular unleaded', 'premium unleaded (recommended)'],
            "flex_fuel" : ['flex-fuel (unleaded/E85)', 'flex-fuel (premium unleaded recommended/E85)', 'flex-fuel (premium unleaded required/E85)', 'flex-fuel (unleaded/natural gas)']
            }
def fuel_lbl_map(row):
    for k_,v_ in fuel_map.items():
        if row in v_:
            row = k_[:]
    return row

df_cat['Engine Fuel Type'] = df_cat['Engine Fuel Type'].apply(lambda x: fuel_lbl_map(x))


# Consolidate vehicle style categories into broader categories 
vh_sty_map = {"Hatchback" : ['4dr Hatchback', '2dr Hatchback'],
               "Minivan" : ['Passenger Minivan', 'Cargo Minivan'],
               "SUV" : ['4dr SUV', '2dr SUV', 'Convertible SUV'],
               "Pickup" :['Crew Cab Pickup', 'Regular Cab Pickup', 'Extended Cab Pickup'],
               "Van" : ['Cargo Van','Passenger Van']
             }

def style_lbl_map(row):
    for k_,v_ in vh_sty_map.items():
        if row in v_:
            row = k_[:]
    return row
    
df_cat['Vehicle Style'] = df_cat['Vehicle Style'].apply(lambda x: style_lbl_map(x))


# function to replace market categories with broader labels given above 
mk_cats_lst = ['Crossover', 'Diesel', 'Exotic', 'Factory Tuner', 'Flex Fuel', 'Hatchback', 'Luxury,']
def map_market_cat(row):
    for cat_ in mk_cats_lst:
        if cat_ in row: 
            row = cat_[:]
    return row
    
# apply the function and do further cleanup 
df_cat['Market Category'] = df_cat['Market Category'].apply(lambda x: map_market_cat(x))
df_cat['Market Category'] = df_cat['Market Category'].str.replace('Luxury,', 'Luxury')
df_cat['Market Category'] = df_cat['Market Category'].str.replace('Performance,Hybrid', 'Hybrid')

# we remove the model column due its 915 unique values - will add large no. of feature columns post one hot encoding 
drop_cols = ['Model']
df_cat.drop(drop_cols, axis = 'columns' , inplace = True)                         

In [ ]:
print("Unique categories post-consolidation:")
for col_ in df_cat.columns:
    print("{}:{}".format(col_,df_cat[col_].nunique()))

In [ ]:
# use one hot encoder to encode categorical label
encoder = OneHotEncoder(sparse=False, drop='first')
ohe = encoder.fit_transform(df_cat)
df_ohe = pd.DataFrame(ohe, columns=encoder.get_feature_names_out(df_cat.columns))

# concat with the numerical df to get final dataframe 
df_fin = pd.concat([df_num_fin, df_ohe], axis=1)

### Regression Model

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import KFold, cross_val_score

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor,StackingRegressor

from sklearn.metrics import mean_squared_error,r2_score

In [ ]:
# define the features and target data 
X  = df_fin.drop(trgt_col, axis = 'columns')
y = df_fin[trgt_col]


# split the dataset into training and testing 
X_train,X_test,y_train, y_test = train_test_split(X,y, test_size = 0.20, random_state = 123) 

# print the shape of the split datasets 
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

# scale the training and testing data 
sclr = StandardScaler()
X_train = sclr.fit_transform(X_train)
X_test = sclr.transform(X_test)

In [ ]:
# fit the model on the training data and predict values for the test data 
y_pred_dt,r2_dt = dict(),dict()

# Simple Linear Regression 
lr = LinearRegression()
lr.fit(X_train,y_train)
y_pred_dt['linear'] = lr.predict(X_test)
r2_dt['linear'] = lr.score(X_test, y_test).round(2)


# Ridge Regression (with L2 Regularization) 
alpha_ridge = 1
rr = Ridge(alpha = alpha_ridge)
rr.fit(X_train, y_train)
y_pred_dt['ridge'] = rr.predict(X_test)
r2_dt['ridge'] = rr.score(X_test, y_test).round(2)

# LASSO Regression (with L1 Regularization)
alpha_lass = 5
lass = Lasso(alpha = alpha_lass)
lass.fit(X_train, y_train)
y_pred_dt['lasso'] = lass.predict(X_test)
r2_dt['lasso'] = lass.score(X_test, y_test).round(2)

# Elastic Net Regularization (Combine L1 + L2) 
en = ElasticNet(alpha = 1, l1_ratio = 0.5, random_state =123)
en.fit(X_train,y_train)
y_pred_dt['elastic_net'] = en.predict(X_test)
r2_dt['elastic_net'] = en.score(X_test, y_test).round(2)


# Also trying ensemble method to assess predictability vs interpretability dilemma (Ensemble methods being more complex)  
# will use Random Forest Regressor 
rf = RandomForestRegressor(n_estimators = 40, max_depth = None, bootstrap = True, random_state=123)
rf_pp = make_pipeline(rf)

rf_pp.fit(X_train, y_train)
y_pred_dt['Random_Forest'] = rf_pp.predict(X_test)
r2_dt['Random_Forest'] =r2_score(y_test, y_pred_dt['Random_Forest']).round(2)

In [ ]:
# Summarize the model performance across different algorithms 
df_res = pd.DataFrame(columns = ['Model Type', 'RMSE', 'R-squared Value'])

for key,value in y_pred_dt.items():
    mse = mean_squared_error(y_test,y_pred_dt[key])
    rmse = np.round(math.sqrt(mse),2)
    df_res.loc[len(df_res)] = [key, rmse, r2_dt[key]]

In [ ]:
# Create a k-fold cross-validation object
kf_cv_res = list()
k_folds = 4
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

for model in [lr,rr, lass,en, rf]:
    cross_val_accuracy = cross_val_score(model, X, y, cv=kf, scoring='r2')
    kf_cv_res.append(cross_val_accuracy.mean().round(2))

# concat k-fold cv results to results df 
df_res['k_fold_mean_R2'] = kf_cv_res
df_res

### Conclusion 


We observe that Random Forest gives a high model score of 95% while also reducing the mean squared error the most. Though it presents a challenge of becoming a black-box model i.e. low intepretability of random forest structure, it is significantly more effective in achieving more accurate predictions.

In [ ]:
# ## Optional:
# # Try to further improve Random Forest performance by hyperparameter tuning 
# param_grid = {
#     'n_estimators': [50, 100, 200],
#     'max_depth': [None, 10, 20, 30],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4],
# }

# model = RandomForestRegressor() 

# grid_search = GridSearchCV(model, param_grid, cv=5, scoring='r2', n_jobs=-1)
# grid_search.fit(X_train, y_train)

# best_params = grid_search.best_params_
# best_model = grid_search.best_estimator_